In [1]:
import json
import logging
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss
import os
from os.path import join
import sys
import pickle
import csv
from transformers import BertTokenizer,BertForSequenceClassification
from transformers.optimization import AdamW

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.set_device(6)

In [3]:
# set dimension of interest
# dim = 'romance'
dims = [
        'social_support',
        'conflict',
        'trust',
        'fun',
        'similarity',
        'identity',
        'respect',
        'romance',
        'knowledge',
        'power'
        ]

In [4]:
# DATA_DIR = 'data/%s'%dim # directory of where the train/test/dev files are stored
# OUTPUT_DIR = 'weights/BERT/%s' %dim # where the model weights will be stored
BERT_MODEL = 'bert-base-cased' # BERT model type
CACHE_DIR = 'cache/' # where BERT will look for pre-trained models to load parameters from

num_labels = 2

OUTPUT_MODE = 'classification'
CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [5]:
# load model
model = BertForSequenceClassification.from_pretrained(BERT_MODEL,cache_dir=CACHE_DIR, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import json
## load sentences
file = open('C:\\Users\\fonta\\PycharmProjects\\10_dimensions_classifier\\out\\reddit_samples.txt')
comments = json.load(file)
scoring = {}
for id in comments:
    sentence = comments[id]
    scoring.update({sentence: {}})
    print(f'\n{id}: {sentence}')
    for dim in dims:
        DATA_DIR = 'data/%s'%dim
        OUTPUT_DIR = 'weights/BERT/%s' %dim
        # load pretrained BERT model for specific dimension
        output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
        model.load_state_dict(torch.load(output_model_file))
        model.to(device)
        tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False)

        input_ids = torch.tensor([tokenizer.encode(sentence,add_special_tokens=True)]).cuda()
        with torch.no_grad():
            outputs = model(input_ids)[0]
            score = torch.softmax(outputs,1)
        scoring[sentence].update({dim: score[0,1].item()})
        print("\t%s:\t%1.3f" % (dim,score[0,1].item()))

with open('C:\\Users\\fonta\\PycharmProjects\\10_dimensions_classifier\\out\\reddit_scoring.txt', 'w') as convert_file:
     convert_file.write(json.dumps(scoring))


e5jcml5: You seem like you thought on this for a little too long...
	social_support:	0.077
	conflict:	0.013
	trust:	0.025
	fun:	0.025
	similarity:	0.061
	identity:	0.003
	respect:	0.011
	romance:	0.001
	knowledge:	0.003
	power:	0.011

e5hxhex: It evolved to adapt to the crushing pressure and oppressive darkness.
	social_support:	0.009
	conflict:	0.343
	trust:	0.028
	fun:	0.025
	similarity:	0.212
	identity:	0.008
	respect:	0.025
	romance:	0.000
	knowledge:	0.801
	power:	0.011

e5a5d7h: There goes my theory that everyone who's blonde is a royal ^^/s
	social_support:	0.013
	conflict:	0.757
	trust:	0.026
	fun:	0.025
	similarity:	0.004
	identity:	0.004
	respect:	0.001
	romance:	0.001
	knowledge:	0.510
	power:	0.011

e5y8n53: Because the thin blue line protects itself first and foremost. 
	social_support:	0.123
	conflict:	0.015
	trust:	0.027
	fun:	0.025
	similarity:	0.046
	identity:	0.032
	respect:	0.006
	romance:	0.001
	knowledge:	0.978
	power:	0.011

e6wfpic: They’re known for chocking ag